<a href="https://colab.research.google.com/github/shivkaansh/burnout/blob/master/shiv_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [230]:
import numpy as np # you know why we used
import pandas as pd # dataframe kisme karenge fir ye use nai kiya to
from sklearn.metrics import r2_score 

In [231]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [232]:
# Import the dataset

# Apni apni file ka path dedo

train_data = pd.read_csv("/content/drive/MyDrive/train_data/train.csv") 
test_data = pd.read_csv("/content/drive/MyDrive/train_data/test.csv")

print(train_data.shape)
print(test_data.shape)

(22750, 9)
(12250, 8)


In [233]:
train_data.head()

,Employee ID,Date of Joining,Gender,Company Type,WFH Setup Available,Designation,Resource Allocation,Mental Fatigue Score,Burn Rate
0,fffe32003000360033003200,2008-09-30,Female,Service,No,2.0,3.0,3.8,0.16
1,fffe3700360033003500,2008-11-30,Male,Service,Yes,1.0,2.0,5.0,0.36
2,fffe31003300320037003900,2008-03-10,Female,Product,Yes,2.0,NaN,5.8,0.49
3,fffe32003400380032003900,2008-11-03,Male,Service,Yes,1.0,1.0,2.6,0.20
4,fffe31003900340031003600,2008-07-24,Female,Service,No,3.0,7.0,6.9,0.52


In [234]:
train_data.dtypes

Employee ID              object
Date of Joining          object
Gender                   object
Company Type             object
WFH Setup Available      object
Designation             float64
Resource Allocation     float64
Mental Fatigue Score    float64
Burn Rate               float64
dtype: object

In [235]:
# check the null values in test data
null_values = test_data.isnull().sum()
print(null_values)

Employee ID             0
Date of Joining         0
Gender                  0
Company Type            0
WFH Setup Available     0
Designation             0
Resource Allocation     0
Mental Fatigue Score    0
dtype: int64


In [236]:
# check the null values in train data data
train_null_values = train_data.isnull().sum()
print(train_null_values)

Employee ID                0
Date of Joining            0
Gender                     0
Company Type               0
WFH Setup Available        0
Designation                0
Resource Allocation     1381
Mental Fatigue Score    2117
Burn Rate               1124
dtype: int64


In [237]:
# Null Values removing
train_data.dropna(axis=0,inplace=True, how='all', subset= ["Burn Rate"])

In [238]:
# Changing the dtypes of DOJ, Gender, Companytype
# DOJ

# getting the month from data of join
train_data["Date of Joining"] = pd.to_datetime(train_data["Date of Joining"])
test_data["Date of Joining"] = pd.to_datetime(test_data["Date of Joining"])

train_data["Month"] = train_data["Date of Joining"].dt.month
test_data["Month"] = test_data["Date of Joining"].dt.month

train_data.drop(labels=["Date of Joining"], axis=1, inplace=True)
test_data.drop(labels=["Date of Joining"], axis=1, inplace=True)

In [239]:
# Gender
# replacing female with 0 and male with 1

train_data["Gender"].replace({'Female':0,'Male':1}, inplace=True)
test_data["Gender"].replace({'Female':0,'Male':1}, inplace=True)

In [240]:
# Employee ID unique
print("Unique Employee ID in training set ", train_data["Employee ID"].nunique())
print("Unique Employee ID in training set ",test_data["Employee ID"].nunique())

Unique Employee ID in training set  21626
Unique Employee ID in training set  12250


In [241]:
# Company type 

# replacing service with 0 and product 1
print("Unique Values in Company Type column in training set" , train_data["Company Type"].unique())
print("Unique Values in Company Type column in Test Set",test_data["Company Type"].unique())

train_data["Company Type"].replace({'Service':0,'Product':1}, inplace=True)
test_data["Company Type"].replace({'Service':0,'Product':1}, inplace=True)

Unique Values in Company Type column in training set ['Service' 'Product']
Unique Values in Company Type column in Test Set ['Service' 'Product']


In [242]:
# WFH Setup Available

# replacing service with 0 and product 1
print("Unique Values in WFH column in training set" , train_data["WFH Setup Available"].unique())
print("Unique Values in WFH column in Test Set",test_data["WFH Setup Available"].unique())

train_data["WFH Setup Available"].replace({'No':0,'Yes':1}, inplace=True)
test_data["WFH Setup Available"].replace({'No':0,'Yes':1}, inplace=True)

Unique Values in WFH column in training set ['No' 'Yes']
Unique Values in WFH column in Test Set ['No' 'Yes']


In [243]:
# changing designation datatype
train_data["Designation"] = train_data["Designation"].astype(np.int32)
test_data["Designation"] = test_data["Designation"].astype(np.int32)

In [244]:
train_data.dtypes

Employee ID              object
Gender                    int64
Company Type              int64
WFH Setup Available       int64
Designation               int32
Resource Allocation     float64
Mental Fatigue Score    float64
Burn Rate               float64
Month                     int64
dtype: object

In [245]:
train_data.head()

,Employee ID,Gender,Company Type,WFH Setup Available,Designation,Resource Allocation,Mental Fatigue Score,Burn Rate,Month
0,fffe32003000360033003200,0,0,0,2,3.0,3.8,0.16,9
1,fffe3700360033003500,1,0,1,1,2.0,5.0,0.36,11
2,fffe31003300320037003900,0,1,1,2,NaN,5.8,0.49,3
3,fffe32003400380032003900,1,0,1,1,1.0,2.6,0.20,11
4,fffe31003900340031003600,0,0,0,3,7.0,6.9,0.52,7


In [246]:
X_train = train_data.drop(labels=["Employee ID", "Burn Rate","Month"], axis=1)
X_test = test_data.drop(labels=["Employee ID","Month"], axis=1)
y_train = train_data["Burn Rate"]

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)

(21626, 6)
(12250, 6)
(21626,)


In [251]:
# Training a model
! pip install catboost
from catboost import CatBoostRegressor
cat_boost = CatBoostRegressor(iterations=500, learning_rate=0.5,)
cat_boost.fit(X_train, y_train)

y_pred_train = cat_boost.predict(X_train)
y_pred_test = cat_boost.predict(X_test)
print("Prediction on training set ", 100*r2_score(y_train, y_pred_train))

0:	learn: 0.1264287	total: 4.76ms	remaining: 2.37s
1:	learn: 0.0932897	total: 8.45ms	remaining: 2.1s
2:	learn: 0.0764686	total: 12.1ms	remaining: 2s
3:	learn: 0.0689364	total: 15.7ms	remaining: 1.95s
4:	learn: 0.0656134	total: 20ms	remaining: 1.98s
5:	learn: 0.0644145	total: 23.8ms	remaining: 1.96s
6:	learn: 0.0635168	total: 27.5ms	remaining: 1.94s
7:	learn: 0.0629951	total: 30.9ms	remaining: 1.9s
8:	learn: 0.0625613	total: 34.4ms	remaining: 1.88s
9:	learn: 0.0622598	total: 38.3ms	remaining: 1.88s
10:	learn: 0.0619280	total: 42ms	remaining: 1.87s
11:	learn: 0.0614942	total: 45.9ms	remaining: 1.87s
12:	learn: 0.0612091	total: 49.5ms	remaining: 1.85s
13:	learn: 0.0608426	total: 53.1ms	remaining: 1.84s
14:	learn: 0.0607027	total: 56.6ms	remaining: 1.83s
15:	learn: 0.0604990	total: 60.3ms	remaining: 1.82s
16:	learn: 0.0603710	total: 64.2ms	remaining: 1.82s
17:	learn: 0.0601402	total: 67.7ms	remaining: 1.81s
18:	learn: 0.0599698	total: 71.3ms	remaining: 1.8s
19:	learn: 0.0598862	total: 75.3

In [252]:
submit = pd.DataFrame()
submit['Employee ID'] = test_data['Employee ID']

In [253]:
submit["Burn Rate"] = y_pred_test

In [254]:
submit.head()

,Employee ID,Burn Rate
0,fffe31003300390039003000,0.620274
1,fffe31003300310037003800,0.351886
2,fffe33003400380035003900,0.441575
3,fffe3100370039003200,0.381566
4,fffe32003600390036003700,0.534085


In [255]:
submit.to_csv('/content/drive/MyDrive/train_data/sample_submission.csv',index=False)